In [5]:
import random
import string

In [6]:
class Candidate(object):
    def __init__(self, genotype, target):
        self.genotype = list(genotype)
        self.target = target

    def fitness(self):
        fitval = 0
        for i in range(0, len(self.genotype)):
            fitval += (ord(self.target[i]) - ord(self.genotype[i])) ** 2
        return(fitval)

    def mutate(self, probability=0.1, distance=1):
        for pos in range(len(self.genotype)):
            if random.random() < probability:
                self.genotype[pos] = chr(
                    ord(self.genotype[pos]) + random.randint(-distance, distance)
                )

    def crossover(self, partner):
        start = random.randint(0, len(partner.genotype) - 1)
        end = random.randint(0, len(partner.genotype) -1)
        if start > end:
            stop, end = start, end
        child_genotype = self.genotype[:]
        child_genotype[start:end] = partner.genotype[start:end]
        return(self.__class__(child_genotype, self.target))

    def __repr__(self):
        return('%7i  %s' % (self.fitness(), ''.join(self.genotype)))


In [7]:
class Population(object):
    def __init__(self, candidate, target, size=20, max_generations=5000):
        self.candidate = candidate
        self.size = size
        self.target = target
        self.generation = 0
        self.population = self.gen_population(self.size)
        self.max_generations = max_generations

    def evolve(self):
        while True:
            self.population.sort(key = lambda candidate: candidate.fitness())

            if self.population[0].fitness() == 0:
                return(self.generation)

            if self.generation == self.max_generations:
                return(False)

            self.generation += 1

            rnd1 = int(random.random() * random.random() * (self.size -1))
            rnd2 = int(random.random() * random.random() * (self.size -1))
            parent1 = self.population[rnd1]
            parent2 = self.population[rnd2]
            child = parent1.crossover(parent2)
            child.mutate()

            if child.fitness() < self.population[-1].fitness():
                self.population[-1] = child

    def gen_candidate(self, target):
        return(
            self.candidate(
                [random.choice(string.printable[:-5]) for j in range(len(target))],
                target
            )
        )

    def gen_population(self, size):
        population = []
        for i in range(size):
            population.append(self.gen_candidate(self.target))
        return(population)

    def __repr__(self):
        return(
            '\n'.join(
                ['%7i  %7i  %s' % (self.generation, c.fitness(), ''.join(c.genotype)) for c in self.population]
            )
        )


In [8]:
def run(target, probability, distance):
    class CustomCandidate(Candidate):
        def mutate(self, probability=probability, distance=distance):
            return(Candidate.mutate(self, probability, distance))

    population = Population(CustomCandidate, target)
    generations = population.evolve()

    tmpl = "Target %s (probability=%f, distance=%i) within %i generations"
    if not generations:
        tmpl = tmpl % ("not reached", probability, distance, population.generation)
    else:
        tmpl = tmpl % ("reached", probability, distance, population.generation)
    print(tmpl)

In [9]:
run('Hello, World!', 0.1, 1)

Target reached (probability=0.100000, distance=1) within 2560 generations
